# Utils

**Импорты библиотек:**

In [1]:
!pip install tqdm==4.66.5
!pip install torch==2.4.1+cu121
!pip install torchvision==0.19.1+cu121
!pip install transformers==4.44.2
!pip install comet_ml==3.47.6
!pip install triton==3.1.0
!pip install torchattacks==3.5.1
!pip install jax==0.4.38
!pip install jaxlib==0.4.38
!pip install pytorch_optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [3]:
import random

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os

import comet_ml

import torch
import torchvision
from torchvision.transforms import v2
import torchattacks
from transformers import BeitConfig, BeitForMaskedImageModeling
import torch.nn as nn

from transformers import AdamW
import torch.optim as optim
from transformers import Dinov2ForImageClassification
from transformers import AutoModel
from copy import deepcopy

**Utils:**

**Функция для фиксирования сида:**

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**Additional utils:**

In [5]:
def get_root(env):
    # supports env == "colab", env == "kaggle"
    if env == "kaggle":
        return "/kaggle/working/"
    else:
        return "./"

In [7]:
# this augmentation first converts PIL image to Tensor,
# then casts resulting tensor to type=torch.float32,
# finally normalizes the image

default_aug = v2.Compose([v2.PILToTensor(),
                          v2.ToDtype(torch.float32, scale=True),
                          v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [8]:
# This function is used later in DataLoaders to fix the seed setting bugs when using several workers

def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed() % (2**32)
    random.seed(torch_seed)
    np.random.seed(torch_seed)

**Датасет + DataLoaders:**

In [9]:
def get_CIFAR10_data(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    print("Prepare Test Set:  ✅")
    return data_train, data_test

In [10]:
def get_CIFAR10_dataloaders(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Test Set:  ✅")
    return dataloader_train, dataloader_test

**Код цикла предобучения/дообучения SSL-метода:**

In [11]:

def pretrain_beit(model, processor, train_loader, epochs, optimizer, scaler, scheduler=None, device="cuda"):
    model.train()
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}")
        for batch_idx, (images, _) in enumerate(train_loader):
            images = images.to(device, memory_format=torch.channels_last)

            inputs = processor(images, return_tensors="pt", do_rescale=False).to(device)
            pixel_values = inputs["pixel_values"]
            
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(pixel_values=pixel_values, labels=labels)
                loss = outputs.loss
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            if batch_idx % 100 == 0:
                print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")
        
        if scheduler:
            scheduler.step()
        print(f"Epoch {epoch} completed. Current LR: {scheduler.get_last_lr()[0]}")
    
    print("Pretraining Complete!")
    return model

In [12]:
def pretrain(model, train_loader, optimizer, criterion, epochs=10):
  model.train()
  for epoch in range(epochs):
      running_loss = 0.0
      for images, labels in train_loader:
          images = images.to(device)
          labels = labels.to(device)

          # Zero gradients
          optimizer.zero_grad()

          # Extract features using pre-trained DINOv2 backbone
          with torch.no_grad():
              features = model.dino(images)

          # Forward pass through the custom classifier head
          outputs = model.classifier(features)

          # Calculate the loss
          loss = criterion(outputs, labels)

          # Backpropagate the loss and optimize
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      # Print loss for the epoch
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

In [13]:
comet_ml.login()

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
NUM_EPOCHS = 50
BATCH_SIZE = 128
exp_name = "BEiT_Pretrain"

In [16]:
from transformers import BeitForImageClassification, BeitConfig, AutoImageProcessor, BeitForMaskedImageModeling

In [17]:
# # random seed
# seed_everything(0)

# # device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# #model
# beit_config = BeitConfig.from_pretrained("microsoft/beit-base-patch16-224")
# model = BeitForMaskedImageModeling.from_pretrained(
#     "microsoft/beit-base-patch16-224", config=beit_config
# )
# model.to(device)

# optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
# scaler = torch.cuda.amp.GradScaler()
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224")

# # pretrained_model = pretrain_beit(model, processor, train_loader, epochs=NUM_EPOCHS, optimizer=optimizer, scaler=scaler, scheduler=scheduler, device=device)

# train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE)

# torch.save(pretrained_model.state_dict(), "beit_cifar10_pretrained.pth")
# print("Pretrained model saved!")

**Downstream-модель:**

In [18]:
class DownstreamModel(torch.nn.Module):
    def __init__(self, encoder, head):
        super(DownstreamModel, self).__init__()
        self.encoder = encoder
        self.head = head

    def forward(self, x):
        enc_features = self.encoder(x)
        logits = self.head(enc_features)
        return logits

**Код для обычного/adversarial Fine-Tuning'а (на одном GPU):**

In [19]:
def eval_epoch(model, loader, attack=None):
    val_loss = 0.
    val_acc = 0.

    model.eval()

    for data, target in tqdm(loader):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if attack is not None:
            # perform adversarial attack
            with torch.enable_grad():
                data = attack(data, target)
        
        logits = model(data)
        loss = torch.nn.functional.cross_entropy(logits, target)

        val_loss += (loss.item() * data.shape[0])
        val_acc += (torch.argmax(logits, dim=1) == target).sum().item()

    val_loss /= len(loader.dataset)
    val_acc /= len(loader.dataset)
    return val_loss, val_acc


def train_epoch(model, optimizer, loader, scaler, epoch, exp, is_attack_used=False):
    train_loss = 0.
    train_acc = 0.

    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, target) in enumerate(tqdm(loader)):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if is_attack_used != "no":
            if is_attack_used == "BIM":
                attack = torchattacks.BIM(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            else:
                attack = torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=20)
            # perform adversarial attack
            data = attack(data, target)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            logits = model(data)
            loss = torch.nn.functional.cross_entropy(logits, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            train_loss += loss.item() * data.shape[0]
            train_acc += (torch.argmax(logits, dim=1) == target).sum().item()

        if exp is not None:
            exp.log_metrics({"train loss (by steps)": loss.item()}, step=(step + batch_idx))

    train_loss /= len(loader.dataset)
    train_acc /= len(loader.dataset)
    return train_loss, train_acc, scaler


def train(model, exp_name, optimizer, epochs, train_loader, val_loader, is_train_attack_used="no", scheduler=None, exp=None, env="colab"):
    best_val_acc = 0.
    best_val_acc_epoch = None
    best_model_state_filepath = None

    val_attacks_names = ["No Attack"]

    scaler = torch.GradScaler("cuda")

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        metrics = dict()
        
        train_loss, train_acc, scaler = train_epoch(model, optimizer, train_loader, scaler, epoch, exp, is_attack_used=is_train_attack_used)

        val_attacks = [None]
        
        for it in range(len(val_attacks)):
            val_loss, val_acc = eval_epoch(model, val_loader, attack=val_attacks[it])
            metrics[val_attacks_names[it]] = {"Loss": val_loss, "Accuracy": val_acc}

        if scheduler is not None:
            scheduler.step()

        print("\n")
        print(" Train:")
        print(f"  Loss: {train_loss}, Accuracy: {train_acc}")
        print(" Validation:")
        for it in metrics.items():
            print(f"  Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


        if exp is not None:
            exp.log_metrics(
                   {"train loss": train_loss,
                    "train accuracy": train_acc},
                    step=(epoch * len(train_loader)))
            for it in metrics.items():
                exp.log_metrics(
                    {f"validation loss ({it[0]})": it[1]['Loss'],
                     f"validation accuracy ({it[0]})": it[1]['Accuracy']},
                     step=(epoch * len(train_loader)))

        val_acc = metrics["No Attack"]["Accuracy"]
        
        if val_acc > best_val_acc:
            print("🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉")
            best_val_acc = val_acc
            best_val_acc_epoch = epoch
            filepath = f"{get_root(env)}best_weights_{exp_name}_epoch{best_val_acc_epoch}.pth"
            torch.save(model.state_dict(), filepath)
            print("Save State of the Best Model: ✅")
            if best_model_state_filepath is not None:
                os.remove(best_model_state_filepath)
                print("Delete State File of the Previous Best Model: ✅")
            best_model_state_filepath = filepath

        print('-----------------------------------------------------------------\n')

    if exp is not None:
        exp.log_model(name=best_model_state_filepath[len(get_root(env)):-4],
                      file_or_folder=best_model_state_filepath)


def test(model, loader, env="colab"):
    attacks_names = ["No Attack", "i-FGSM", "PGD", "DI-FGSM"]
    
    metrics = dict()

    attacks = [None, 
               torchattacks.BIM(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.DIFGSM(model, eps=(8 / 255), alpha=(2 / 255), steps=10, resize_rate=0.9, diversity_prob=0.5)
              ]
    
    for it in range(len(attacks)):
        loss, acc = eval_epoch(model, loader, attack=attacks[it])
        metrics[attacks_names[it]] = {"Loss": loss, "Accuracy": acc}

    print("\n")
    print("Final Metrics:")
    for it in metrics.items():
        print(f" {it[0]}:")
        print(f"   Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")

**Примерный код, как с помощью весов получить энкодер модели + создание downstream-модели (нужно для SSL-методов):**

# Эксперименты

In [20]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
comet_ml.login()

**DINOV2 (Adversarial SFT with i-FGSM)**


In [22]:
from transformers import Dinov2Config, Dinov2Model, AutoImageProcessor
import torch.nn as nn

from transformers import AdamW

In [24]:
class DINOV2Classifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super(DINOV2Classifier, self).__init__()
        self.encoder = encoder

        head_hidden_dim = self.encoder.config.hidden_size * 2

        self.head = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, head_hidden_dim),
            nn.GELU(),
            nn.Linear(head_hidden_dim, num_classes)
        )

    def forward(self, x):
        enc_features = self.encoder(x).last_hidden_state[:, 0, :]
        logits = self.head(enc_features)
        return logits

In [27]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "DINOV2_SFT_IFGSM"

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

dino_config = Dinov2Config.from_pretrained("facebook/dinov2-base")
dino_config.image_size = 32
dino_config.patch_size = 8
encoder = Dinov2Model.from_pretrained("facebook/dinov2-base", config=dino_config)


model = DINOV2Classifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

In [192]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [194]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="BIM", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : DINOV2_SFT_IFGSM
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial1/05c7474eefc44e81957d5ab2f926fb0f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss                      : 158101.0
COMET INFO:     train loss (by steps) [6] : (2.3474197387695312, 2.5756378173828125)
COMET INFO:   Others:
COMET INFO:     Name : DINOV2_SFT_IFGSM
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model graph         : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO:    

Epoch 1:



100%|██████████| 79/79 [00:06<00:00, 13.10it/s]




 Train:
  Loss: 2.1608295459747313, Accuracy: 0.1764
 Validation:
  Loss: 1.8820790790557862, Accuracy: 0.3158
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:06<00:00, 13.09it/s]


...

100%|██████████| 79/79 [00:06<00:00, 12.83it/s]




 Train:
  Loss: 1.8055519774627686, Accuracy: 0.32454
 Validation:
  Loss: 1.558454284286499, Accuracy: 0.4454
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:06<00:00, 12.91it/s]




 Train:
  Loss: 1.7584461963272096, Accuracy: 0.34148
 Validation:
  Loss: 1.4968596673965453, Accuracy: 0.4725
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : DINOV2_SFT_IFGSM
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial1/7ce29ed2d74f499ebc9be295fdeb0eed
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (108032.125, 155632.5)
COMET INFO:     train accuracy [5]                  : (0.1764, 0.34148)
COMET INFO:     train loss (by steps) [1955]        : (1.5514507293701172, 2.3747634887695312)
COMET INFO:     train loss [5]                      : (1.7584461963272096, 2.1608295459747313)
COMET INFO:     validation accuracy (No Attack) [5] : (0.3158, 0.4725)
COMET INFO:     validation loss (No Attack) [5]     : (1.49685966739

In [196]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [01:50<00:00,  1.40s/it]



Final Metrics:
 No Attack:
   Loss: 1.4968596673965453, Accuracy: 0.4725
 i-FGSM:
   Loss: 2.0498110645294187, Accuracy: 0.2289
 PGD:
   Loss: 2.0485941862106323, Accuracy: 0.2284
 DI-FGSM:
   Loss: 1.98732036151886, Accuracy: 0.2611


In [199]:
ifgsm_dinov2_model = model

**DINOV2 (Adversarial SFT with PGD)**

In [201]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "DINOV2_SFT_PGD"

In [202]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

dino_config = Dinov2Config.from_pretrained("facebook/dinov2-base")
dino_config.image_size = 32
dino_config.patch_size = 8
encoder = Dinov2Model.from_pretrained("facebook/dinov2-base", config=dino_config)

model = DINOV2Classifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)

DINOV2Classifier(
  (encoder): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2Attention(
            (attention): Dinov2SelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
        

In [203]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [204]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="PGD", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial1/1f7837ba09e248439d00a37390f3389c

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:06<00:00, 13.07it/s]




 Train:
  Loss: 2.181662226409912, Accuracy: 0.16822
 Validation:
  Loss: 1.9456262552261352, Accuracy: 0.2696
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:06<00:00, 13.06it/s]


...

100%|██████████| 79/79 [00:06<00:00, 13.10it/s]




 Train:
  Loss: 1.8289044873428344, Accuracy: 0.31388
 Validation:
  Loss: 1.5413680648803711, Accuracy: 0.4535
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:06<00:00, 12.98it/s]




 Train:
  Loss: 1.7776247240447998, Accuracy: 0.33182
 Validation:
  Loss: 1.5006690692901612, Accuracy: 0.4627
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : DINOV2_SFT_PGD
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial1/1f7837ba09e248439d00a37390f3389c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (106012.625, 157085.5)
COMET INFO:     train accuracy [5]                  : (0.16822, 0.33182)
COMET INFO:     train loss (by steps) [1955]        : (1.5195541381835938, 2.64935302734375)
COMET INFO:     train loss [5]                      : (1.7776247240447998, 2.181662226409912)
COMET INFO:     validation accuracy (No Attack) [5] : (0.2696, 0.4627)
COMET INFO:     validation loss (No Attack) [5]     : (1.500669069290161

In [206]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [01:50<00:00,  1.40s/it]



Final Metrics:
 No Attack:
   Loss: 1.5006690692901612, Accuracy: 0.4627
 i-FGSM:
   Loss: 2.077979367637634, Accuracy: 0.2272
 PGD:
   Loss: 2.0766576553344724, Accuracy: 0.2275
 DI-FGSM:
   Loss: 2.0136199920654296, Accuracy: 0.2567


In [207]:
pgd_dinov2_model = model

**DINOV2 (Adversarial SFT with i-FGSM, without patch resizing)**


In [36]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "DINOV2_SFT_IFGSM_DEFAULT_BATCHES"

In [39]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

dino_config = Dinov2Config.from_pretrained("facebook/dinov2-base")
encoder = Dinov2Model.from_pretrained("facebook/dinov2-base", config=dino_config)

model = DINOV2Classifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

In [40]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [41]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="BIM", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial1/982c8607671b4886aae0f127af4c1ca3

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:02<00:00, 31.89it/s]




 Train:
  Loss: 1.7943931985473633, Accuracy: 0.30772
 Validation:
  Loss: 1.2567140504837035, Accuracy: 0.5364
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:02<00:00, 32.04it/s]


...

100%|██████████| 79/79 [00:02<00:00, 31.98it/s]




 Train:
  Loss: 0.6405138680839538, Accuracy: 0.7633
 Validation:
  Loss: 0.8155667651176453, Accuracy: 0.7194
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:02<00:00, 31.44it/s]




 Train:
  Loss: 0.3805852573394775, Accuracy: 0.8612
 Validation:
  Loss: 0.8532550748825073, Accuracy: 0.7422
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : DINOV2_SFT_IFGSM_DEFAULT_BATCHES
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial1/982c8607671b4886aae0f127af4c1ca3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (7359.841796875, 168509.0)
COMET INFO:     train accuracy [5]                  : (0.30772, 0.8612)
COMET INFO:     train loss (by steps) [1955]        : (0.2018156796693802, 2.6139450073242188)
COMET INFO:     train loss [5]                      : (0.3805852573394775, 1.7943931985473633)
COMET INFO:     validation accuracy (No Attack) [5] : (0.5364, 0.7422)
COMET INFO:     validation loss (No Attack) [5] 

In [42]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [00:40<00:00,  1.94it/s]



Final Metrics:
 No Attack:
   Loss: 0.8532550748825073, Accuracy: 0.7422
 i-FGSM:
   Loss: 1.699885546875, Accuracy: 0.5554
 PGD:
   Loss: 3.639106824493408, Accuracy: 0.1659
 DI-FGSM:
   Loss: 3.455557440185547, Accuracy: 0.19


In [43]:
ifgsm_dinov2_model_def = model

**DINOV2 (Adversarial SFT with PGD, without patch resizing)**

In [48]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "DINOV2_SFT_PGD_DEFAULT_BATCHES"

In [49]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

dino_config = Dinov2Config.from_pretrained("facebook/dinov2-base")
encoder = Dinov2Model.from_pretrained("facebook/dinov2-base", config=dino_config)

model = DINOV2Classifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

In [50]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [51]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="PGD", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial1/4ac0bb4bec694ee08fd7778992af2007



Epoch 1:



COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
100%|██████████| 79/79 [00:02<00:00, 28.37it/s]




 Train:
  Loss: 2.2299147218322752, Accuracy: 0.14558
 Validation:
  Loss: 2.0779168338775635, Accuracy: 0.1925
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:02<00:00, 31.12it/s]


...

100%|██████████| 79/79 [00:02<00:00, 32.04it/s]




 Train:
  Loss: 1.9097110302352904, Accuracy: 0.28846
 Validation:
  Loss: 1.8409463247299194, Accuracy: 0.2883
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:02<00:00, 31.00it/s]




 Train:
  Loss: 1.8597099291992187, Accuracy: 0.304
 Validation:
  Loss: 1.782223934173584, Accuracy: 0.3297
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : DINOV2_SFT_PGD_DEFAULT_BATCHES
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial1/4ac0bb4bec694ee08fd7778992af2007
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (14023.21875, 185721.0)
COMET INFO:     train accuracy [5]                  : (0.14558, 0.304)
COMET INFO:     train loss (by steps) [1955]        : (1.6214427947998047, 2.8338775634765625)
COMET INFO:     train loss [5]                      : (1.8597099291992187, 2.2299147218322752)
COMET INFO:     validation accuracy (No Attack) [5] : (0.1925, 0.3297)
COMET INFO:     validation loss (No Attack) [5]     : 

In [52]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [00:40<00:00,  1.94it/s]



Final Metrics:
 No Attack:
   Loss: 1.782223934173584, Accuracy: 0.3297
 i-FGSM:
   Loss: 1.8917817932128906, Accuracy: 0.2903
 PGD:
   Loss: 1.8521169567108153, Accuracy: 0.3155
 DI-FGSM:
   Loss: 1.8441736524581909, Accuracy: 0.3189


In [53]:
pgd_dinov2_model_def = model

**DINOV2 (Adversarial SFT without attacks)**

In [55]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "DINOV2_SFT"

In [56]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

dino_config = Dinov2Config.from_pretrained("facebook/dinov2-base")
encoder = Dinov2Model.from_pretrained("facebook/dinov2-base", config=dino_config)

model = DINOV2Classifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

In [57]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [58]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="no", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial1/d1ea05d8cfaa40459396c87297ca0bee

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:02<00:00, 30.88it/s]




 Train:
  Loss: 1.518885706653595, Accuracy: 0.43746
 Validation:
  Loss: 1.0800968616485596, Accuracy: 0.6112
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:02<00:00, 31.34it/s]


...

100%|██████████| 79/79 [00:02<00:00, 28.23it/s]




 Train:
  Loss: 0.23895977121353149, Accuracy: 0.92042
 Validation:
  Loss: 0.8043508144378662, Accuracy: 0.7627
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:02<00:00, 30.77it/s]




 Train:
  Loss: 0.07539934298992157, Accuracy: 0.97798
 Validation:
  Loss: 0.9349986005783081, Accuracy: 0.7688
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : DINOV2_SFT
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial1/d1ea05d8cfaa40459396c87297ca0bee
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (583.0015258789062, 153564.0)
COMET INFO:     train accuracy [5]                  : (0.43746, 0.97798)
COMET INFO:     train loss (by steps) [1955]        : (0.010935388505458832, 2.6825180053710938)
COMET INFO:     train loss [5]                      : (0.07539934298992157, 1.518885706653595)
COMET INFO:     validation accuracy (No Attack) [5] : (0.6112, 0.7688)
COMET INFO:     validation loss (No Attack) [5]     : (0.8043508

In [59]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [00:40<00:00,  1.94it/s]



Final Metrics:
 No Attack:
   Loss: 0.9349986005783081, Accuracy: 0.7688
 i-FGSM:
   Loss: 4.111857750701905, Accuracy: 0.3629
 PGD:
   Loss: 8.67746005706787, Accuracy: 0.0439
 DI-FGSM:
   Loss: 7.640306941223145, Accuracy: 0.0741
